In [1]:
import struct
import zlib

In [2]:
def imread_png_grayscale(filename):
    with open(filename, 'rb') as f:
        # Verify PNG signature
        signature = f.read(8)
        if signature != b'\x89PNG\r\n\x1a\n':
            raise ValueError("File is not a valid PNG image.")
        
        # Read chunks until we get the IHDR and IDAT chunks
        width = height = None
        bit_depth = color_type = None
        compressed_data = b''
        
        while True:
            # Each chunk has 4 parts: length (4 bytes), type (4 bytes), data, CRC (4 bytes)
            chunk_length = struct.unpack(">I", f.read(4))[0]  # Length of chunk data
            chunk_type = f.read(4)  # Type of chunk
            
            # Read the chunk data
            chunk_data = f.read(chunk_length)
            # Skip CRC
            f.read(4)
            
            if chunk_type == b'IHDR':
                # IHDR chunk contains metadata
                width, height, bit_depth, color_type, _, _, _ = struct.unpack(">IIBBBBB", chunk_data)
                if color_type != 0:
                    raise ValueError("Only grayscale PNG images are supported.")
                
            elif chunk_type == b'IDAT':
                # Collect IDAT data for decompression
                compressed_data += chunk_data
                
            elif chunk_type == b'IEND':
                # IEND marks the end of the file
                break
        
        # Decompress the IDAT data
        decompressed_data = zlib.decompress(compressed_data)
        
        # Convert decompressed data to a 2D array of pixels
        # Each row of the image starts with a filter byte
        image_data = []
        row_size = width
        
        i = 0
        for row in range(height):
            # Skip the filter byte
            filter_type = decompressed_data[i]
            i += 1
            
            # Read each pixel in the row
            row_data = []
            for col in range(row_size):
                row_data.append(decompressed_data[i])
                i += 1
            
            image_data.append(row_data)
        
    return image_data  # Returns a 2D list representing the grayscale image

In [5]:
def flatten_image(image_data):
    # Use a list comprehension to flatten the 2D array into a 1D array
    flattened_image = [pixel for row in image_data for pixel in row]
    return flattened_image

In [ ]:
import os
import random

# Directory path Switch this up for submission
directory_path = 'C:\\Users\\Admin\\coding-projs\\cmsc422\\HW\\422hw3\\facedata\\Face Data for Homework\\ATT'
sorted_files = sorted(os.listdir(directory_path))
label_and_data = [] # [label, data]

# Iterate through each file in the directory
for index, filename in enumerate(sorted_files):
    file_path = os.path.join(directory_path, filename)
    if 'README' in filename:
        continue

    # Check if it's a file (not a directory)
    if os.path.isfile(file_path):
        # Find the part of the filename up to the first underscore
        class_number = filename.split('_', 1)[0]
        flattened_data = flatten_image(imread_png_grayscale(file_path))
        label_and_data.append([[class_number, flattened_data]])

random.shuffle(label_and_data)
print(len(label_and_data))

400


In [60]:
data_for_pca = [data[0][1] for data in label_and_data]

In [24]:
def euclidean_distance(img1, img2):
    # Ensure the images have the same length
    if len(img1) != len(img2):
        raise ValueError("Images must have the same size.")
    
    # Initialize the sum of squared differences
    squared_diff_sum = 0
    
    # Loop over each pixel value in the flattened arrays
    for i in range(len(img1)):
        # Calculate the difference between corresponding pixels
        diff = img1[i] - img2[i]
        
        # Square the difference and add to the sum
        squared_diff_sum += diff ** 2
    
    # Take the square root of the sum to get the Euclidean distance
    distance = squared_diff_sum ** 0.5
    return distance


In [28]:
five_folds = []
fold = []

for entry in label_and_data:
    if len(fold) == 79:
        fold.append(entry)
        five_folds.append([f for f in fold])
        fold = []
    else:
        fold.append(entry)
            

In [43]:
print(len(five_folds))

5


In [54]:
def get_1nn_predictions(train, test):
    amount_correct = 0
    for i in range(len(test)):
        test_me = test[i][0]
        # print(len(test_me))
        real_label = test_me[0]
        test_data = test_me[1]
        pred = [float('inf'), 'none']  # distance, label
        for j in range(len(train)):
            label = train[j][0][0]
            data = train[j][0][1]
            dist = euclidean_distance(data, test_data)
            if dist < pred[0]:
                pred = [dist, label]
        if pred[1] == real_label:
            amount_correct += 1
    print(f"Amount correct {amount_correct}")
    return amount_correct


In [55]:
accuracy  = []

for i in range(len(five_folds)):
    predict_for_me = five_folds[i]
    # print(len(predict_for_me))
    ammount_correct = 0
    for j in range(len(five_folds)):
        if i == j:
            continue
        test_on_me = five_folds[j]
        ammount_correct += get_1nn_predictions(test_on_me, predict_for_me)
    print(f"between fold {i} and {j} there were {ammount_correct} correct predictions")
    accuracy.append((ammount_correct / 320) * 100)

print( f"Total accuracy {sum(accuracy) / 5}" )
        

Amount correct 57
Amount correct 49
Amount correct 39
Amount correct 53
between fold 0 and 4 there were 198 correct predictions
Amount correct 53
Amount correct 58
Amount correct 45
Amount correct 48
between fold 1 and 4 there were 204 correct predictions
Amount correct 56
Amount correct 57
Amount correct 53
Amount correct 52
between fold 2 and 4 there were 218 correct predictions
Amount correct 56
Amount correct 43
Amount correct 52
Amount correct 50
between fold 3 and 4 there were 201 correct predictions
Amount correct 57
Amount correct 54
Amount correct 53
Amount correct 41
between fold 4 and 4 there were 205 correct predictions
Total accuracy 64.125


The printed statements here are misleading. That is not the number of correct predictions between fold i and j. It is the number of correct predictions with fold i as the test set

In [56]:
one_NN_accuracy = {sum(accuracy) / 5} 
print(f"one NN accuracy {one_NN_accuracy}")

one NN accuracy {64.125}


In [73]:
def get_five_folds(data):
    five_folds = []
    fold = []
    print(len(data), "data")
    for entry in data:
        if len(fold) == 79:
            fold.append(entry)
            five_folds.append([f for f in fold])
            fold = []
        else:
            fold.append(entry)
    return five_folds

In [92]:
def revamped_1nn(train, test):
    amount_correct = 0
    for i in range(len(test)):
        test_me = test[i][0]
        # print(len(test_me))
        real_label = test_me[0]
        test_data = test_me[1]
        pred = [float('inf'), 'none']  # distance, label
        for j in range(len(train)):
            label = train[j][0][0]
            data = train[j][0][1]
            dist = euclidean_distance(data, test_data)
            if dist < pred[0]:
                pred = [dist, label]
        if pred[1] == real_label:
            amount_correct += 1
    print(f"Amount correct {amount_correct}")
    return amount_correct

In [ ]:
def get_kfold_results(five_folds):
    accuracy  = []

    for i in range(len(five_folds)):
        predict_for_me = five_folds[i]
        # print(len(predict_for_me))
        ammount_correct = 0
        for j in range(len(five_folds)):
            if i == j:
                continue
            test_on_me = five_folds[j]

            ammount_correct += revamped_1nn(test_on_me, predict_for_me)
        print(f"between fold {i} and {j} there were {ammount_correct} correct predictions")
        accuracy.append((ammount_correct / 320) * 100)
    one_NN_accuracy = {sum(accuracy) / 5} 
    return one_NN_accuracy
        

In [57]:
# Step 1: Define functions for basic matrix operations

# Function to compute the mean of each column (feature)
def compute_mean(data):
    mean_vector = [sum(col) / len(col) for col in zip(*data)]
    return mean_vector

# Function to center the data by subtracting the mean vector
def center_data(data, mean_vector):
    centered_data = [[x - mean for x, mean in zip(row, mean_vector)] for row in data]
    return centered_data

# Function to transpose a matrix
def transpose(matrix):
    return [list(row) for row in zip(*matrix)]

# Function to compute covariance matrix
def covariance_matrix(data):
    n = len(data)
    d = len(data[0])
    cov_matrix = [[0] * d for _ in range(d)]
    for i in range(d):
        for j in range(d):
            cov_matrix[i][j] = sum(data[k][i] * data[k][j] for k in range(n)) / (n - 1)
    return cov_matrix

# Function to multiply two matrices
def matrix_multiply(A, B):
    rows_A = len(A)
    cols_A = len(A[0])
    cols_B = len(B[0])
    result = [[0] * cols_B for _ in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_B):
            result[i][j] = sum(A[i][k] * B[k][j] for k in range(cols_A))
    return result

# PCA function
def pca(data, num_components):
    # Step 2: Standardize the data
    mean_vector = compute_mean(data)
    centered_data = center_data(data, mean_vector)

    # Step 3: Calculate the covariance matrix
    cov_matrix = covariance_matrix(centered_data)

    # Step 4: Calculate eigenvalues and eigenvectors (using a library function for this part)
    import numpy as np  # ONLY for eigen decomposition
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

    # Step 5: Sort the eigenvalues and corresponding eigenvectors in descending order
    sorted_indices = sorted(range(len(eigenvalues)), key=lambda k: eigenvalues[k], reverse=True)
    sorted_eigenvectors = [eigenvectors[:, i] for i in sorted_indices]

    # Step 6: Select the top `num_components` eigenvectors
    selected_eigenvectors = [sorted_eigenvectors[i] for i in range(num_components)]

    # Step 7: Project the centered data onto the new feature space
    # Converting eigenvectors to list format
    selected_eigenvectors = [[vec[i] for vec in selected_eigenvectors] for i in range(len(selected_eigenvectors[0]))]
    projected_data = matrix_multiply(centered_data, selected_eigenvectors)

    return projected_data, selected_eigenvectors, mean_vector


In [62]:

# Example usage
# Assuming each 112x96 image is flattened to a 1D array of length 10752 (112 * 96)
# # and we have 'n' images stored in a list of lists 'data' where each sublist is a flattened image
# n = 100  # Example number of images
# d = 112 * 96  # Dimension of each image
# data = [[(i + j) % 256 for j in range(d)] for i in range(n)]  # Replace with your actual data matrix

# Let's say we want to reduce to 50 principal components
num_components = 100
projected_data, principal_components, mean_vector = pca(data_for_pca, num_components)

print("Projected Data Shape:", len(projected_data), len(projected_data[0]) if projected_data else 0)
print("Principal Components Shape:", len(principal_components), len(principal_components[0]) if principal_components else 0)


Projected Data Shape: 400 100
Principal Components Shape: 10304 100


In [79]:
data_for_prediction = []
print(len(projected_data))
for i in range(len(label_and_data)):
    data_sample_with_label = label_and_data[i][0] #[label, data]
    pca_data_sample = projected_data[i]
    # print(len(pca_data_sample))
    data_for_prediction.append([data_sample_with_label[0], pca_data_sample])


400


In [100]:
print(len(data_for_prediction[0][1]))
print(len(label_and_data[0][0][1]))


100
10304


In [93]:
five_fold = get_five_folds(data_for_prediction)
print(len(five_fold))
pca_plus_1nn_accuracy = get_kfold_results(five_folds)
print(f"avg accuracy with pca and one nn are {pca_plus_1nn_accuracy}")

400 data
5
2
1
Amount correct 57
1
Amount correct 49
1
Amount correct 39
1
Amount correct 53
between fold 0 and 4 there were 198 correct predictions
2
1
Amount correct 53
1
Amount correct 58
1
Amount correct 45
1
Amount correct 48
between fold 1 and 4 there were 204 correct predictions
2
1
Amount correct 56
1
Amount correct 57
1
Amount correct 53
1
Amount correct 52
between fold 2 and 4 there were 218 correct predictions
2
1
Amount correct 56
1
Amount correct 43
1
Amount correct 52
1
Amount correct 50
between fold 3 and 4 there were 201 correct predictions
2
1
Amount correct 57
1
Amount correct 54
1
Amount correct 53
1
Amount correct 41
between fold 4 and 4 there were 205 correct predictions
avg accuracy with pca and one nn are {64.125}


In [101]:
import numpy as np

def cumulative_variance(data_matrix, num_components=100):
    # Step 1: Center the data
    mean_vector = np.mean(data_matrix, axis=0)
    centered_data = data_matrix - mean_vector
    
    # Step 2: Compute the covariance matrix
    covariance_matrix = np.cov(centered_data, rowvar=False)

    # Step 3: Calculate eigenvalues
    eigenvalues, _ = np.linalg.eigh(covariance_matrix)

    # Step 4: Sort eigenvalues in descending order
    sorted_eigenvalues = np.sort(eigenvalues)[::-1]

    # Step 5: Calculate explained variance ratio for each component
    total_variance = np.sum(sorted_eigenvalues)
    explained_variance_ratio = sorted_eigenvalues / total_variance

    # Step 6: Compute cumulative variance for the top `num_components`
    cumulative_variance_explained = np.sum(explained_variance_ratio[:num_components])

    print(f"Cumulative variance explained by the top {num_components} components: {cumulative_variance_explained * 100:.2f}%")

# Example usage
# Assuming `data_matrix` is your matrix with each row as a flattened image
cumulative_variance(data_for_pca, num_components=100)


Cumulative variance explained by the top 100 components: 89.06%
